# Notebook 3: Pump-and-Dump Manipulation

Este notebook explora esquemas de **pump-and-dump**, uma das formas mais comuns de manipulação de mercado.

## Objetivos de Aprendizado

1. Entender a mecânica de ataques pump-and-dump
2. Simular as três fases: accumulation, pump, dump
3. Analisar o impacto na riqueza do manipulador vs outros traders
4. Identificar padrões detectáveis no preço e volume

In [ ]:
# Imports
import random
import numpy as np
from market_lab.core.market import MarketConfig
from market_lab.core.traders import build_traders
from market_lab.core.sentiment import NoSentiment
from market_lab.core.simulation import SimulationRunner
from market_lab.manipulation.manipulator import Manipulator
from market_lab.viz.plots import plot_price_series, plot_volume_series

import matplotlib.pyplot as plt
%matplotlib inline

## Anatomia de um Pump-and-Dump

Um ataque pump-and-dump tem **três fases**:

### 1. Accumulation (Acumulação)
- Manipulador compra secretamente grandes quantidades
- Usa preços ligeiramente abaixo do mercado
- Objetivo: acumular holdings sem alertar o mercado

### 2. Pump (Inflação)
- Manipulador cria volume artificial comprando e vendendo para si mesmo
- Preço sobe artificialmente
- Outros traders veem o movimento e entram (FOMO)

### 3. Dump (Despejo)
- Manipulador vende todas as holdings acumuladas
- Preço colapsa
- Traders comuns ficam com prejuízo
- Manipulador lucra

## Configuração do Mercado

Vamos criar um mercado com wealth limits e um manipulador rico.

In [ ]:
# Configuração do mercado
config = MarketConfig(
    n_traders=150,
    initial_price=100.0,
    price_volatility=2.0,
    max_daily_volume=10.0,
    wealth_mode="limited",
    price_tick=1.0,
    seed=42
)

# Criar traders normais
rng = random.Random(42)
traders = build_traders(config, rng)

# Criar manipulador com capital 10x maior
manipulator = Manipulator(
    trader_id="manipulator",
    rng=random.Random(123),
    wealth=100_000.0,  # 10x mais rico que traders normais
    holdings=50.0,
    accumulation_days=30,  # 30 dias acumulando
    pump_days=10,          # 10 dias de pump
    dump_days=15           # 15 dias de dump
)

print(f"Mercado configurado:")
print(f"  Traders normais: {len(traders)}")
print(f"  Manipulador: wealth=${manipulator.wealth:,.0f}, holdings={manipulator.holdings}")
print(f"\nFases do ataque:")
print(f"  Dias 0-29: Accumulation")
print(f"  Dias 30-39: Pump")
print(f"  Dias 40+: Dump")

## Executar Simulação com Manipulador

In [ ]:
# Executar simulação
runner = SimulationRunner(
    config=config,
    traders=traders,
    sentiment=NoSentiment(),
    manipulator=manipulator
)

states = runner.run(n_days=100)
print(f"Simulação concluída: {len(states)} dias")

## Visualização: Preço e Volume

Vamos identificar as três fases visualmente.

In [ ]:
# Visualizar preço e volume com marcadores de fase
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Preço
days = [s.day for s in states]
prices = [s.price for s in states]
ax1.plot(days, prices, linewidth=2, color='blue')
ax1.set_ylabel('Preço', fontsize=12)
ax1.set_title('Pump-and-Dump: Preço ao longo do tempo', fontsize=14, fontweight='bold')
ax1.grid(alpha=0.3)

# Marcar fases
ax1.axvspan(0, 29, alpha=0.2, color='green', label='Accumulation')
ax1.axvspan(30, 39, alpha=0.2, color='orange', label='Pump')
ax1.axvspan(40, 99, alpha=0.2, color='red', label='Dump')
ax1.legend(loc='upper left')

# Volume
volumes = [s.volume for s in states]
ax2.bar(days, volumes, color='steelblue', alpha=0.7)
ax2.set_xlabel('Dia', fontsize=12)
ax2.set_ylabel('Volume', fontsize=12)
ax2.set_title('Volume de Negociação', fontsize=14)
ax2.grid(alpha=0.3)

# Marcar fases
ax2.axvspan(0, 29, alpha=0.2, color='green')
ax2.axvspan(30, 39, alpha=0.2, color='orange')
ax2.axvspan(40, 99, alpha=0.2, color='red')

plt.tight_layout()
plt.show()

# Análise de preço por fase
print("\nAnálise de Preço por Fase:")
print(f"\nAccumulation (dias 0-29):")
print(f"  Preço inicial: {prices[0]:.2f}")
print(f"  Preço final: {prices[29]:.2f}")
print(f"  Variação: {((prices[29] - prices[0]) / prices[0] * 100):.2f}%")

print(f"\nPump (dias 30-39):")
print(f"  Preço inicial: {prices[30]:.2f}")
print(f"  Preço pico: {max(prices[30:40]):.2f}")
print(f"  Variação: {((max(prices[30:40]) - prices[30]) / prices[30] * 100):.2f}%")

print(f"\nDump (dias 40+):")
print(f"  Preço pré-dump: {prices[39]:.2f}")
print(f"  Preço final: {prices[-1]:.2f}")
print(f"  Queda: {((prices[-1] - prices[39]) / prices[39] * 100):.2f}%")

## Análise de Riqueza: Manipulador vs Traders

Vamos rastrear a riqueza total do manipulador e compará-la com traders normais.

In [ ]:
# Simular novamente rastreando riqueza
# Reset manipulator and traders
manipulator_tracker = Manipulator(
    trader_id="manipulator",
    rng=random.Random(123),
    wealth=100_000.0,
    holdings=50.0,
    accumulation_days=30,
    pump_days=10,
    dump_days=15
)

traders_tracker = build_traders(config, random.Random(42))

runner_tracker = SimulationRunner(
    config=config,
    traders=traders_tracker,
    sentiment=NoSentiment(),
    manipulator=manipulator_tracker
)

states_tracker = runner_tracker.run(n_days=100)

# Rastrear riqueza ao longo do tempo
manip_wealth_history = []
avg_trader_wealth_history = []

for state in states_tracker:
    # Riqueza do manipulador = cash + holdings * price
    manip_total_wealth = manipulator_tracker.wealth + (manipulator_tracker.holdings * state.price)
    manip_wealth_history.append(manip_total_wealth)
    
    # Riqueza média dos traders normais
    trader_wealths = [t.wealth + (t.holdings * state.price) for t in traders_tracker]
    avg_trader_wealth_history.append(np.mean(trader_wealths))

# Visualizar comparação de riqueza
plt.figure(figsize=(14, 6))
plt.plot(days, manip_wealth_history, linewidth=2.5, label='Manipulador', color='red')
plt.plot(days, avg_trader_wealth_history, linewidth=2.5, label='Traders (média)', color='blue')
plt.xlabel('Dia', fontsize=12)
plt.ylabel('Riqueza Total ($)', fontsize=12)
plt.title('Comparação de Riqueza: Manipulador vs Traders', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(alpha=0.3)

# Marcar fases
plt.axvspan(0, 29, alpha=0.1, color='green')
plt.axvspan(30, 39, alpha=0.1, color='orange')
plt.axvspan(40, 99, alpha=0.1, color='red')

plt.tight_layout()
plt.show()

# Calcular lucro do manipulador
initial_wealth = 100_000.0 + (50.0 * prices[0])
final_wealth = manip_wealth_history[-1]
profit = final_wealth - initial_wealth
roi = (profit / initial_wealth) * 100

print(f"\nResultados Financeiros do Manipulador:")
print(f"  Riqueza inicial: ${initial_wealth:,.2f}")
print(f"  Riqueza final: ${final_wealth:,.2f}")
print(f"  Lucro: ${profit:,.2f}")
print(f"  ROI: {roi:.2f}%")

# Comparar com traders normais
initial_avg = 10_000.0  # Aproximado
final_avg = avg_trader_wealth_history[-1]
avg_change = ((final_avg - initial_avg) / initial_avg) * 100

print(f"\nTraders Normais (média):")
print(f"  Riqueza inicial: ~${initial_avg:,.2f}")
print(f"  Riqueza final: ${final_avg:,.2f}")
print(f"  Variação: {avg_change:.2f}%")

## Detecção de Manipulação

Quais padrões podem revelar o pump-and-dump?

In [ ]:
# Calcular métricas de detecção
returns = [(prices[i] - prices[i-1]) / prices[i-1] * 100 for i in range(1, len(prices))]

# 1. Spike em retornos
return_zscore = [(r - np.mean(returns)) / np.std(returns) for r in returns]

# 2. Volume anormal
volume_mean = np.mean(volumes)
volume_std = np.std(volumes)
volume_zscore = [(v - volume_mean) / volume_std for v in volumes]

# Visualizar sinais de detecção
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 12))

# Preço
ax1.plot(days, prices, linewidth=2)
ax1.set_ylabel('Preço')
ax1.set_title('Sinais de Detecção de Pump-and-Dump', fontweight='bold')
ax1.grid(alpha=0.3)
ax1.axvspan(0, 29, alpha=0.1, color='green')
ax1.axvspan(30, 39, alpha=0.1, color='orange')
ax1.axvspan(40, 99, alpha=0.1, color='red')

# Z-score de retornos
ax2.plot(days[1:], return_zscore, linewidth=2, color='purple')
ax2.axhline(y=3, color='red', linestyle='--', label='Threshold (±3σ)')
ax2.axhline(y=-3, color='red', linestyle='--')
ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
ax2.set_ylabel('Return Z-Score')
ax2.set_title('Anomalia em Retornos')
ax2.legend()
ax2.grid(alpha=0.3)
ax2.axvspan(0, 29, alpha=0.1, color='green')
ax2.axvspan(30, 39, alpha=0.1, color='orange')
ax2.axvspan(40, 99, alpha=0.1, color='red')

# Z-score de volume
ax3.plot(days, volume_zscore, linewidth=2, color='orange')
ax3.axhline(y=3, color='red', linestyle='--', label='Threshold (±3σ)')
ax3.axhline(y=-3, color='red', linestyle='--')
ax3.axhline(y=0, color='gray', linestyle='-', alpha=0.3)
ax3.set_xlabel('Dia')
ax3.set_ylabel('Volume Z-Score')
ax3.set_title('Anomalia em Volume')
ax3.legend()
ax3.grid(alpha=0.3)
ax3.axvspan(0, 29, alpha=0.1, color='green')
ax3.axvspan(30, 39, alpha=0.1, color='orange')
ax3.axvspan(40, 99, alpha=0.1, color='red')

plt.tight_layout()
plt.show()

# Identificar dias com anomalias
anomaly_days = [i for i in range(len(return_zscore)) 
                if abs(return_zscore[i]) > 3 or volume_zscore[i+1] > 3]
print(f"\nDias com anomalias detectadas: {anomaly_days}")
print(f"Total de anomalias: {len(anomaly_days)} dias")

## Padrões Característicos de Pump-and-Dump

### Sinais Detectáveis:

1. **Fase de Pump**:
   - Retornos positivos extremos (>3σ)
   - Volume anormalmente alto
   - Velocidade de subida não sustentável

2. **Fase de Dump**:
   - Retornos negativos extremos (<-3σ)
   - Queda abrupta após pico
   - Volume alto seguido de colapso

3. **Assimetria**:
   - Subida gradual (accumulation)
   - Spike rápido (pump)
   - Queda ainda mais rápida (dump)

### Por que funciona?

- **Assimetria de informação**: Manipulador sabe suas intenções, outros não
- **FOMO (Fear of Missing Out)**: Traders veem subida e entram sem análise
- **Timing perfeito**: Manipulador controla quando entrar e sair

## Exercícios Práticos

### Exercício 1: Diferentes Durações
Modifique a duração das fases:
- O que acontece com pump muito curto (5 dias)?
- E se accumulation for muito longa (60 dias)?
- Como isso afeta a detectabilidade?

### Exercício 2: Capital do Manipulador
Teste com diferentes níveis de riqueza:
- Manipulador com wealth=50,000 (5x traders normais)
- Manipulador com wealth=200,000 (20x traders normais)
- Como o capital afeta o lucro?

### Exercício 3: Número de Traders
Compare mercados com diferentes liquidez:
- 50 traders (baixa liquidez)
- 300 traders (alta liquidez)
- Pump-and-dump é mais fácil em mercados ilíquidos?

### Exercício 4: Detectar sem conhecer as fases
Implemente um detector que:
- Não sabe quando as fases começam/terminam
- Apenas observa preço e volume
- Alerta quando detecta padrão suspeito

**Dica**: Use rolling windows e thresholds adaptativos

In [ ]:
# Exemplo de solução para Exercício 4: Detector Simples
def detect_pump_and_dump(prices, volumes, window=10, threshold=2.5):
    """
    Detector simples baseado em spikes de preço e volume.
    """
    alerts = []
    
    for i in range(window, len(prices)):
        # Calcular retorno e volume recente
        recent_return = (prices[i] - prices[i-1]) / prices[i-1] * 100
        window_prices = prices[i-window:i]
        window_volumes = volumes[i-window:i]
        
        # Z-scores
        return_mean = np.mean([(window_prices[j] - window_prices[j-1]) / window_prices[j-1] * 100 
                               for j in range(1, len(window_prices))])
        return_std = np.std([(window_prices[j] - window_prices[j-1]) / window_prices[j-1] * 100 
                             for j in range(1, len(window_prices))])
        volume_mean = np.mean(window_volumes)
        volume_std = np.std(window_volumes)
        
        if return_std > 0:
            return_z = (recent_return - return_mean) / return_std
        else:
            return_z = 0
            
        if volume_std > 0:
            volume_z = (volumes[i] - volume_mean) / volume_std
        else:
            volume_z = 0
        
        # Alerta se ambos forem anormais
        if abs(return_z) > threshold and volume_z > threshold:
            alerts.append({
                'day': i,
                'return_z': return_z,
                'volume_z': volume_z,
                'type': 'PUMP' if return_z > 0 else 'DUMP'
            })
    
    return alerts

# Aplicar detector
alerts = detect_pump_and_dump(prices, volumes)

print(f"\nAlertas detectados: {len(alerts)}\n")
for alert in alerts:
    print(f"Dia {alert['day']}: {alert['type']} detectado")
    print(f"  Return Z-score: {alert['return_z']:.2f}")
    print(f"  Volume Z-score: {alert['volume_z']:.2f}")
    print()

## Conclusões

Neste notebook, você aprendeu:

1. **Anatomia do pump-and-dump**: Três fases distintas com objetivos específicos
2. **Mecânica da manipulação**: Como manipuladores exploram assimetria de informação
3. **Impacto financeiro**: Manipuladores lucram às custas de traders comuns
4. **Padrões detectáveis**: Spikes em retornos e volume revelam manipulação
5. **Detectores práticos**: Métodos estatísticos podem identificar padrões suspeitos

### Implicações Éticas

- Pump-and-dump é **ilegal** na maioria das jurisdições
- Causa danos reais a investidores inocentes
- Reguladores usam técnicas similares para detectar fraudes

### Próximos Passos

No **Notebook 4**, vamos explorar técnicas forenses mais sofisticadas para detectar e quantificar manipulação, incluindo ROC curves e comparação de múltiplos detectores.